# Nestkast Occupancy Model Training - EMSN2

Train een MobileNetV2 model voor nestkast bezetting detectie.

**Features:**
- Data augmentatie (rotatie, kleur, flip, etc.)
- Transfer learning van ImageNet
- Mixed precision training (sneller op Colab GPU)

**Data:**
- Leeg: daglicht beelden van alle nestkasten
- Bezet: nachtbeelden met Koolmees (IR camera)

**Auteur:** EMSN2 / Claude Code

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Configuratie - pas aan naar jouw Drive pad
DRIVE_BASE = '/content/drive/MyDrive/EMSN/nestbox-training'
DATA_DIR = f'{DRIVE_BASE}/data'
OUTPUT_DIR = f'{DRIVE_BASE}/models'

# Check data
import os
print(f"Leeg beelden: {len(os.listdir(f'{DATA_DIR}/leeg'))}")
print(f"Bezet beelden: {len(os.listdir(f'{DATA_DIR}/bezet'))}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
from PIL import Image
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Data Augmentatie

Uitgebreide augmentatie voor robuustheid:
- **Rotatie**: -15° tot +15° (camera kan scheef hangen)
- **Kleurvariatie**: brightness, contrast, saturation (daglicht vs IR)
- **Flip**: horizontaal (vogel kan beide kanten op zitten)
- **Affine**: kleine schaal/translatie variaties
- **Grayscale**: soms IR, soms kleur

In [ ]:
# Hyperparameters
INPUT_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 30
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4
TRAIN_SPLIT = 0.8

# Uitgebreide augmentatie voor training
train_transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE + 32, INPUT_SIZE + 32)),  # Iets groter voor crop
    transforms.RandomCrop(INPUT_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),  # Camera kan scheef hangen
    transforms.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1),  # Kleine verschuiving
        scale=(0.9, 1.1)       # Kleine schaalvariatie
    ),
    transforms.ColorJitter(
        brightness=0.3,   # Variatie in belichting
        contrast=0.3,     # IR vs daglicht contrast
        saturation=0.3,   # Kleurverzadiging
        hue=0.1           # Kleine kleurverschuiving
    ),
    transforms.RandomGrayscale(p=0.2),  # Soms IR-achtig
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),  # Lichte blur
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1))  # Random occlusion
])

# Simpele transform voor validatie
val_transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Transforms gedefinieerd met uitgebreide augmentatie")

In [ ]:
class NestboxDataset(Dataset):
    """Dataset voor nestkast beelden"""
    
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.classes = ['leeg', 'bezet']
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = self.data_dir / class_name
            if class_dir.exists():
                for img_path in class_dir.glob('*.jpg'):
                    self.samples.append((img_path, self.class_to_idx[class_name]))
        
        print(f"Dataset: {len(self.samples)} beelden geladen")
        for c in self.classes:
            count = sum(1 for _, label in self.samples if label == self.class_to_idx[c])
            print(f"  - {c}: {count}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# Laad dataset
full_dataset = NestboxDataset(DATA_DIR, transform=None)  # Transform later

# Split in train/val
train_size = int(TRAIN_SPLIT * len(full_dataset))
val_size = len(full_dataset) - train_size
train_indices, val_indices = random_split(
    range(len(full_dataset)), 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"\nTrain: {len(train_indices)}, Validation: {len(val_indices)}")

In [ ]:
class SubsetWithTransform(Dataset):
    """Subset met aparte transform"""
    def __init__(self, dataset, indices, transform):
        self.dataset = dataset
        self.indices = list(indices)
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[self.indices[idx]]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


# Maak train/val datasets met juiste transforms
train_dataset = SubsetWithTransform(full_dataset, train_indices, train_transform)
val_dataset = SubsetWithTransform(full_dataset, val_indices, val_transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

## Visualiseer Augmentatie

In [ ]:
# Toon voorbeelden van augmentatie
def show_augmented_samples(dataset, n_samples=4, n_augments=4):
    """Toon origineel en geaugmenteerde versies"""
    fig, axes = plt.subplots(n_samples, n_augments + 1, figsize=(15, 3*n_samples))
    
    # Inverse normalisatie voor visualisatie
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    
    indices = np.random.choice(len(full_dataset), n_samples, replace=False)
    
    for i, idx in enumerate(indices):
        img_path, label = full_dataset.samples[idx]
        orig_img = Image.open(img_path).convert('RGB')
        class_name = full_dataset.classes[label]
        
        # Origineel
        axes[i, 0].imshow(orig_img)
        axes[i, 0].set_title(f'Origineel ({class_name})')
        axes[i, 0].axis('off')
        
        # Augmentaties
        for j in range(n_augments):
            aug_img = train_transform(orig_img)
            aug_img = inv_normalize(aug_img)
            aug_img = torch.clamp(aug_img, 0, 1)
            axes[i, j+1].imshow(aug_img.permute(1, 2, 0))
            axes[i, j+1].set_title(f'Aug {j+1}')
            axes[i, j+1].axis('off')
    
    plt.tight_layout()
    plt.show()

show_augmented_samples(full_dataset)

## Model Setup

In [ ]:
def create_model(num_classes=2):
    """MobileNetV2 met custom classifier"""
    # Pretrained MobileNetV2
    model = models.mobilenet_v2(weights='IMAGENET1K_V1')
    
    # Freeze early layers (optioneel - kan uitgecomment worden voor full fine-tuning)
    for param in model.features[:10].parameters():
        param.requires_grad = False
    
    # Custom classifier
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(128, num_classes)
    )
    
    return model


model = create_model(num_classes=2)
model = model.to(device)

# Trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")

In [ ]:
# Loss en optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

# Mixed precision voor snellere training
scaler = torch.cuda.amp.GradScaler()

print("Optimizer en scheduler geconfigureerd")

## Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, scaler, device):
    """Train voor één epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Mixed precision forward pass
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Backward pass met scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, 100. * correct / total


def validate(model, loader, criterion, device):
    """Valideer model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, 100. * correct / total

In [ ]:
# Training
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0
best_model_state = None

print("Start training...")
print("=" * 60)

for epoch in range(NUM_EPOCHS):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    # Update scheduler
    scheduler.step(val_loss)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        marker = ' ⭐ BEST'
    else:
        marker = ''
    
    print(f"Epoch {epoch+1:2d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.1f}% | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.1f}%{marker}")

print("=" * 60)
print(f"Training voltooid! Beste validatie accuracy: {best_val_acc:.1f}%")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy
axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['val_acc'], label='Validation')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Model Opslaan

In [ ]:
# Laad beste model
model.load_state_dict(best_model_state)

# Maak output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save model
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
model_filename = f'nestbox_model_{timestamp}.pt'
model_path = f'{OUTPUT_DIR}/{model_filename}'

torch.save({
    'model_state_dict': model.state_dict(),
    'classes': ['leeg', 'bezet'],
    'input_size': INPUT_SIZE,
    'best_val_acc': best_val_acc,
    'epochs_trained': NUM_EPOCHS,
    'augmentation': 'extensive',
    'training_date': timestamp
}, model_path)

print(f"Model opgeslagen: {model_path}")
print(f"Beste validatie accuracy: {best_val_acc:.1f}%")

# Kopieer ook als 'latest'
import shutil
shutil.copy(model_path, f'{OUTPUT_DIR}/nestbox_model_latest.pt')
print(f"Gekopieerd naar: {OUTPUT_DIR}/nestbox_model_latest.pt")

## Test Model

In [ ]:
# Test op enkele beelden
def predict_and_show(model, dataset, n_samples=8):
    """Voorspel en toon resultaten"""
    model.eval()
    
    fig, axes = plt.subplots(2, n_samples//2, figsize=(15, 6))
    axes = axes.flatten()
    
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    
    indices = np.random.choice(len(dataset), n_samples, replace=False)
    
    for i, idx in enumerate(indices):
        image, label = dataset[idx]
        true_class = full_dataset.classes[label]
        
        # Predict
        with torch.no_grad():
            output = model(image.unsqueeze(0).to(device))
            probs = torch.softmax(output, dim=1)
            pred_idx = probs.argmax(1).item()
            confidence = probs[0, pred_idx].item()
        
        pred_class = full_dataset.classes[pred_idx]
        
        # Show
        img_show = inv_normalize(image)
        img_show = torch.clamp(img_show, 0, 1)
        axes[i].imshow(img_show.permute(1, 2, 0))
        
        color = 'green' if pred_class == true_class else 'red'
        axes[i].set_title(f'True: {true_class}\nPred: {pred_class} ({confidence:.0%})', color=color)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

predict_and_show(model, val_dataset)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Verzamel alle voorspellingen
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['leeg', 'bezet'],
            yticklabels=['leeg', 'bezet'])
plt.xlabel('Voorspeld')
plt.ylabel('Werkelijk')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['leeg', 'bezet']))

## Download Model

Na training kun je het model downloaden naar je Pi via rclone of direct kopiëren.

In [ ]:
print(f"\n{'='*60}")
print("TRAINING VOLTOOID")
print(f"{'='*60}")
print(f"\nModel opgeslagen in: {OUTPUT_DIR}")
print(f"Beste accuracy: {best_val_acc:.1f}%")
print(f"\nKopieer naar Pi met:")
print(f"  rclone copy gdrive:EMSN/nestbox-training/models/nestbox_model_latest.pt /mnt/nas-birdnet-archive/nestbox/models/")